In [14]:
import flywheel
import random
import pandas as pd
import os
import datetime 
fw = flywheel.Client()
collection_id = '60eca24fcfdec364fd96c874' # Neuroprint Validation 

In [2]:
collection_sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [3]:
# create list of subject and session labels
subject_labels = []
session_labels = []
for session in collection_sessions:
    subject = fw.get(session.parents['subject'])
    subject_labels.append(subject.label)
    session_labels.append(session.label)


In [4]:
print(session_labels[0:5])
print(subject_labels[0:5])

['123194x20190603x3T', '119851x20190613x3T', '121105x20170427x3T', '121195x20170523x3T', '121299x20170808x3T']
['123194', '119851', '121105', '121195', '121299']


In [5]:
# stage 1:raw T1ws. Generate filenames
stage1 = ['x'] * len(session_labels)
for i in range(len(session_labels)):
    # generate a pseudorandom positive integer based on the session label 
    rand = abs(hash(session_labels[i]))
    stage1[i] = str(rand)[0:4] # cut number of digits

stage1[0:5]

['5210', '4292', '4310', '3093', '3702']

In [6]:
# stage 2 and 3: orientation and first heatmap label round
numbers = list(range(1, len(session_labels) + 1))
stage23 = numbers.copy()
random.shuffle(stage23)
stage23[0:5]

[25, 43, 9, 47, 7]

In [7]:
# stage 4: second heatmap label round (with clinical context)
stage4 = numbers.copy()
random.shuffle(stage4)
stage4[0:5]

[49, 7, 18, 10, 1]

In [8]:
data = list(zip(subject_labels, session_labels, stage1, stage23, stage4))
df = pd.DataFrame(data, columns=['Subject Label', 'Session Label', 'Stage 1', 'Stages 2 & 3', 'Stage 4'])

In [9]:
df.head()

,Subject Label,Session Label,Stage 1,Stages 2 & 3,Stage 4
0,123194,123194x20190603x3T,5210,25,49
1,119851,119851x20190613x3T,4292,43,7
2,121105,121105x20170427x3T,4310,9,18
3,121195,121195x20170523x3T,3093,47,10
4,121299,121299x20170808x3T,3702,7,1


In [10]:
df.to_csv('lookup_table.csv')

In [15]:
datetime = datetime.datetime.now()
date = str(datetime).replace(' ', '_')
cmd = 'cp lookup_table.csv lookuptable_backups/lookup_table_{}.csv'.format(date)
os.system(cmd)

0